In [1]:
import pandas as pd
import numpy as np
import datetime as dt

## RE is the module for regex . 

In [2]:
import re

## Convert source to dataframe

In [3]:
df = pd.read_json('code_source.py')

Convert string date to datetime 

## Get price by product

In [4]:
df_prod_price = df

In [5]:
df_prod_price = df_prod_price[['transaction_items','transaction_value']]
df_prod_price  = df_prod_price.drop_duplicates(subset = ['transaction_items','transaction_value'])
df_prod_price['transaction_items']
df_prod_price = df_prod_price[~df_prod_price['transaction_items'].str.contains(f";")]

In [6]:
df_prod_price['num'] = df_prod_price['transaction_items'].str.split(',',expand=True)[2]

In [7]:
df_prod_price['transaction_items'] = df_prod_price['transaction_items'].str.split('(',expand=True)[0]

In [8]:
df_prod_price = df_prod_price[df_prod_price['num'] == '(x1)']

In [9]:
df_prod_price.rename(columns ={'transaction_items': 'product_name','transaction_value':'price'},inplace=True)

In [10]:
df_prod_price = df_prod_price[['product_name','price']]

In [11]:
df_prod_price

,product_name,price
3,"HealthyKid 3+,Yummy Vegetables,",500
8,"Candy City,Gummy Worms,",150
55,"Exotic Extras,Beef Chicharon,",1299
61,"Exotic Extras,Kimchi and Seaweed,",799
142,"Candy City,Orange Beans,",199
204,"HealthyKid 3+,Gummy Vitamins,",1500
227,"HealthyKid 3+,Nutrional Milk,",1990


In [ ]:
# df_prod_price[~df_prod_price['transaction_items'].str.contains(f";")]

In [12]:
df['transaction_date'] = pd.to_datetime(df['transaction_date'] )

In [13]:
df['birthdate'] = pd.to_datetime(df['birthdate'] )

check latest date and farthest date of the dataframe

In [14]:
df['month'] = df['transaction_date'].dt.month_name()

In [15]:
df['month_num'] = df['transaction_date'].dt.month

In [16]:
df['transaction_date'].max()

Timestamp('2022-06-30 00:00:00')

In [17]:
df['transaction_date'].min()

Timestamp('2022-01-01 00:00:00')

In [ ]:
# list_[1]

In [ ]:
# inactive_list = df.groupby(['month_num']).username.apply(lambda x: ','.join(x)).fillna('').reset_index().rename({'username': 'previous_list'}, axis=1)

## Transform to Granularity

Assignning the split key for the column transaction_items. 

The current format is 'HealthyKid 3+,Gummy Vitamins,(x3);HealthyKid 3+,Yummy Vegetables,(x2)'
1st step is to split by ";"
2nd step is to split by ","
and get the value with (x3) for the quantity of each item

In [18]:
df = df.assign(var=df['transaction_items'].str.split(';'))

# Convert to Row

In [19]:
df = df.explode('var')

# Repeating the same process above 

In [20]:
df['number_of_purchases'] = df['var'].str.split('(',expand=True)[1].str.replace('x','').str.replace(')','').astype(int)

C:\Users\Kenneth\AppData\Local\Temp\ipykernel_12384\3064442970.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['number_of_purchases'] = df['var'].str.split('(',expand=True)[1].str.replace('x','').str.replace(')','').astype(int)


In [ ]:
# df = df.assign(var=df['var'].str.split(','))

After assigning a split key , df.explode converts the assigned key to rows 

In [ ]:
# df = df.explode('var')

### Remove the frame with value "\\(x" in rows. it should be in new column

# rename assign var column to product name

In [23]:
df.rename(columns ={'var': 'product_name'},inplace=True)

In [24]:
df['product_name'] = df['product_name'].str.split('(',expand=True)[0]

In [25]:
df

,address,birthdate,mail,name,sex,username,transaction_items,transaction_value,transaction_date,month,month_num,product_name,number_of_purchases
0,USNS Wagner\nFPO AE 00917,1926-03-11,watsonbrenda@gmail.com,Jennifer Campbell,F,veronicawells,"Exotic Extras,Beef Chicharon,(x4)",5196,2022-01-12,January,1,"Exotic Extras,Beef Chicharon,",4
1,USCGC Benson\nFPO AP 48794,2016-07-08,moorelisa@gmail.com,Melissa Kim,F,cgilbert,"Exotic Extras,Beef Chicharon,(x3);HealthyKid 3...",12056,2022-01-12,January,1,"Exotic Extras,Beef Chicharon,",3
1,USCGC Benson\nFPO AP 48794,2016-07-08,moorelisa@gmail.com,Melissa Kim,F,cgilbert,"Exotic Extras,Beef Chicharon,(x3);HealthyKid 3...",12056,2022-01-12,January,1,"HealthyKid 3+,Nutrional Milk,",4
1,USCGC Benson\nFPO AP 48794,2016-07-08,moorelisa@gmail.com,Melissa Kim,F,cgilbert,"Exotic Extras,Beef Chicharon,(x3);HealthyKid 3...",12056,2022-01-12,January,1,"Candy City,Orange Beans,",1
2,USCGC Benson\nFPO AP 48794,2016-07-08,moorelisa@gmail.com,Melissa Kim,F,cgilbert,"HealthyKid 3+,Gummy Vitamins,(x3);HealthyKid 3...",5500,2022-01-08,January,1,"HealthyKid 3+,Gummy Vitamins,",3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
77558,"99007 Bautista Shoal\nGonzalezville, NE 86063",1995-05-23,boydmonica@gmail.com,Chase Martin,M,ashleypage,"Exotic Extras,Beef Chicharon,(x2)",2598,2022-06-19,June,6,"Exotic Extras,Beef Chicharon,",2
77559,"99007 Bautista Shoal\nGonzalezville, NE 86063",1995-05-23,boydmonica@gmail.com,Chase Martin,M,ashleypage,"Exotic Extras,Kimchi and Seaweed,(x2)",1598,2022-06-08,June,6,"Exotic Extras,Kimchi and Seaweed,",2
77560,USNV Donovan\nFPO AE 77055,1936-07-02,adam41@hotmail.com,Barbara Berger,F,nbrown,"Exotic Extras,Beef Chicharon,(x3)",3897,2022-06-19,June,6,"Exotic Extras,Beef Chicharon,",3
77561,"422 Bender Crossing\nSouth Dylanbury, DE 09283",1960-11-12,anna66@gmail.com,Chad Mcdonald,M,jessica09,"Exotic Extras,Beef Chicharon,(x1);Candy City,G...",1449,2022-06-26,June,6,"Exotic Extras,Beef Chicharon,",1


### Number of Purchases by Product

 We used the group by to get the summarize number of transactions per month.  Group by in python is like group by in sql

In [26]:
df.columns

Index(['address', 'birthdate', 'mail', 'name', 'sex', 'username',
       'transaction_items', 'transaction_value', 'transaction_date', 'month',
       'month_num', 'product_name', 'number_of_purchases'],
      dtype='object')

In [27]:
df_by_product = df.groupby(['month_num','month','product_name'])['number_of_purchases'].sum().reset_index().sort_values(['month_num','number_of_purchases'],axis=0,ascending= False)

Another group by to get the overall number of purchase by month 

In [28]:
df_by_product = df_by_product.merge(df_prod_price,on=['product_name'],how='left')
#product_price - yung table ng per product and price , dapat same format ng product name 

In [29]:
df_by_product['total_amount'] = df_by_product['price'] * df_by_product['number_of_purchases']
#paggawa new column

In [30]:
df_by_product['total_amount'] = df_by_product['total_amount'].astype(float)

In [31]:
df_by_product

,month_num,month,product_name,number_of_purchases,price,total_amount
0,6,June,"Candy City,Orange Beans,",6216,199,1236984.0
1,6,June,"Exotic Extras,Beef Chicharon,",6159,1299,8000541.0
2,6,June,"Exotic Extras,Kimchi and Seaweed,",6037,799,4823563.0
3,6,June,"HealthyKid 3+,Gummy Vitamins,",5990,1500,8985000.0
4,6,June,"HealthyKid 3+,Yummy Vegetables,",5892,500,2946000.0
5,6,June,"Candy City,Gummy Worms,",5880,150,882000.0
6,6,June,"HealthyKid 3+,Nutrional Milk,",5862,1990,11665380.0
7,5,May,"Exotic Extras,Beef Chicharon,",10028,1299,13026372.0
8,5,May,"Candy City,Orange Beans,",9964,199,1982836.0
9,5,May,"HealthyKid 3+,Gummy Vitamins,",9948,1500,14922000.0


In [ ]:
print(df_by_product)

In [32]:
monthly_count_purchases = df_by_product.groupby(['month_num'])['number_of_purchases'].sum().reset_index()

In [33]:
monthly_count_purchases

,month_num,number_of_purchases
0,1,68041
1,2,69910
2,3,69297
3,4,69257
4,5,69130
5,6,42036


For the transaction value we have to drop the duplicates by transaction date and username, Since we converted the current data to rows. It creates duplicates in trasaction value. The granular Table we made is only applicable to the number of purchases . 

In [34]:
monthly_numbers = df.drop_duplicates(subset = ['transaction_date','username'])

Group by month to get the transaction value by month. 

In [35]:
monthly_numbers = monthly_numbers.groupby(['month','month_num'])['transaction_value'].sum().reset_index().sort_values(['month_num','transaction_value'],ascending=False)

In [36]:
monthly_numbers = monthly_numbers.merge( monthly_count_purchases, how='left', on='month_num')

 Converted the data type of transaction value to float to show decimals

In [37]:
monthly_numbers['transaction_value'] = monthly_numbers['transaction_value'].astype(float)

# Monthly Numbers

In [ ]:
monthly_numbers

## Monthly Repeater

1st We group by the data by month and  convert all the username to a single row by by applying lambda x.join()
2nd The shift(1) , it shifts the value of the month by 1 . 

In [38]:
previous_month_list = df.groupby(['month_num']).username.apply(lambda x: ','.join(x)).shift(1).fillna('').reset_index().rename({'username': 'previous_list'}, axis=1)


In [39]:
previous_month_list

,month_num,previous_list
0,1,
1,2,"veronicawells,cgilbert,cgilbert,cgilbert,cgilb..."
2,3,"veronicawells,veronicawells,veronicawells,cgil..."
3,4,"veronicawells,cgilbert,cgilbert,cgilbert,cgilb..."
4,5,"cgilbert,cgilbert,qhernandez,qhernandez,esmith..."
5,6,"cgilbert,cgilbert,cgilbert,cgilbert,cgilbert,c..."


 We join the above dataframe to our main raw dataframe using pandas merge.
 And apply lambda to determine if the current username exist in previous_list column 

In [40]:
repeater = pd.merge(df, previous_month_list, how='left', on='month_num')
repeater['repeaters'] =repeater.apply(lambda row: row['username'] in row['previous_list'], axis=1)


In [41]:
repeater = repeater.drop_duplicates(subset = ['month','username'])

We applied np.select with the condition of if the column 'repeaters' is true value should be 1


In [42]:
repeater['repeater_count'] = np.select([repeater['repeaters'] == True],[1],default=0)

Finally with the group by to get the monthly numbers by month

In [43]:
repeater = repeater.groupby(['month_num'])['repeater_count'].sum().reset_index()

In [44]:
repeater

,month_num,repeater_count
0,1,0
1,2,5194
2,3,5276
3,4,5229
4,5,5167
5,6,3227


## Monthly Active numbers

drop the duplicates by month and username since we only need to know if they exists in a current month

In [45]:
df_active_engaged = df.drop_duplicates(subset = ['month','username'])

Make a new column count with value of 1 

In [46]:
df_active_engaged['count'] = 1

C:\Users\Kenneth\AppData\Local\Temp\ipykernel_12384\3237565106.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_active_engaged['count'] = 1


First is get the unique values of  month_num column and apply it in loop.
Each value represents the month number of the month 
1 for January 2 for Februany and so on..... 
We used the logic of <= month_num to determine wheter if they exists in current month and previous month

Then we group by by username inside the loop to get if they exists in the current month. 
Then we asign x for column[month_num]

then we append it to the blank list we created. which is frame = []


In [47]:
frame = []
for x in df_active_engaged['month_num'].unique():
    df_ = df_active_engaged[df_active_engaged['month_num']<= x]
    df_ = df_.groupby(['username']).sum().reset_index()
    df_['month_num'] = x
#     df_ = df[['month_num','username','count']]
    frame.append(df_)
                       
    

C:\Users\Kenneth\AppData\Local\Temp\ipykernel_12384\2287691961.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_ = df_.groupby(['username']).sum().reset_index()
C:\Users\Kenneth\AppData\Local\Temp\ipykernel_12384\2287691961.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_ = df_.groupby(['username']).sum().reset_index()
C:\Users\Kenneth\AppData\Local\Temp\ipykernel_12384\2287691961.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be v

We convert the blank frame which has already the data of the loop we created. Using pd.concat it convert the list to dataframe

In [48]:
df_active_engaged = pd.concat(frame)

In [49]:
df_active_engaged = df_active_engaged[['month_num','username','count']]

np.select is applied to determine whether they exists in the current month and the last previous month.
To check if they are active in everymonth, the column count should be equal to the month_num 

In [ ]:
df_active_engaged['engaged'] = np.select([(df_active_engaged['month_num'] - df_active_engaged['count']== 0) & (df_active_engaged['month_num'] != 1)],[1],default=0)

In [ ]:
df_active_engaged = df_active_engaged.groupby(['month_num'])['engaged'].sum().reset_index()

In [ ]:
df_active_engaged

## Inactive List

Drop the duplicate by month_num and username

In [ ]:
df_inactive = df.drop_duplicates(subset = ['month_num','username'])

In [ ]:
df_inactive = df_inactive[['month_num','username']]



We applied same logic from the active number list. 

Get the unique value of month_num column.

Apply it to loop.
x = month_num

First we assign
    df_a as active in the current month. We sliced the frame by the current month_num.
    df_i as the inactive list , we sliced the frame by the month_num < x which is the current month_num
    To avoid duplicate we drop duplicate by username
    Then we check if the username  from the df_i does not exists in df_a(active from prevous months) using "isin" . the operator "~" returns the opposite value of the logic we applied. 
    Then we do the group by
    and comes up with the iactive count by month

In [ ]:
inactive_frame = []
for x in df_inactive['month_num'].unique():
    
    df_a = df_inactive[df_inactive['month_num'] == x]
    df_i = df_inactive[df_inactive['month_num']< x]
    df_i = df_i.drop_duplicates(subset = 'username')
    df_i = df_i[~df_i['username'].isin(df_a['username'])]
    df_i = df_i['username'].count()
    df_a = df_a.groupby(['month_num']).count().reset_index()
    df_a['inactive'] = df_i
    df_a['month_num_'] = x

    
#     df_ = df[['month_num','username','count']]
    inactive_frame.append(df_a)
                       

In [ ]:
inactive_frame = pd.concat(inactive_frame)

In [ ]:
inactive_frame = inactive_frame[['month_num_','inactive','username']]

In [ ]:
inactive_frame.rename(columns ={'month_num_': 'month_num','username':'monthly_customer_count'},inplace=True)

In [ ]:
inactive_frame

# Summary

Merge all the frame we created earlier

In [ ]:
summary = monthly_numbers.merge(repeater,on='month_num',how='left')

In [ ]:
summary = summary.merge(df_active_engaged,on='month_num',how='left')

In [ ]:
summary = summary.merge(inactive_frame,on='month_num',how='left')

In [ ]:
summary = summary.sort_values(['month_num'],ascending=True)

slice the frame and only get the column we needed

In [ ]:
summary = summary[['month', 'transaction_value', 'number_of_purchases',

                   'repeater_count', 'engaged', 'inactive', 'monthly_customer_count']]

In [ ]:
summary

### Slice the frame with the columns we only want to show in the visualization table

In [ ]:
df_by_product = df_by_product[['month','product_name','number_of_purchases','total_amount']]

In [ ]:
df_by_product.rename(columns={'month':'Month','product_name':"Product Name","number_of_purchases":"Number Of Purchases",'total_amount':'Total Amount Sold'},inplace=True)

In [ ]:
fig = ff.create_table(df_by_product)
fig.show()